# **1. Dataset Preparation:**
• Select a time series dataset suitable for forecasting tasks (e.g., stock prices,
weather data, energy consumption).
• Preprocess the dataset, including normalization and splitting into training
and test sets.

In [12]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
     

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [6]:
df = pd.read_csv("weatherdata.csv")

In [7]:
df.head(10)

,Date/Time,Temperature (in Degree Celsius),Dew Point Temperature (in Degree Celsius),Relative Humidity (in %),Wind Speed (in km/h),Visibility (in km),Pressure (in kilopascal),Weather Description
0,1/1/2012 0:00,-1.8,-3.9,86,4,8.0,101.24,Fog
1,1/1/2012 1:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2,1/1/2012 2:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"
3,1/1/2012 3:00,-1.5,-3.2,88,6,4.0,101.27,"Freezing Drizzle,Fog"
4,1/1/2012 4:00,-1.5,-3.3,88,7,4.8,101.23,Fog
5,1/1/2012 5:00,-1.4,-3.3,87,9,6.4,101.27,Fog
6,1/1/2012 6:00,-1.5,-3.1,89,7,6.4,101.29,Fog
7,1/1/2012 7:00,-1.4,-3.6,85,7,8.0,101.26,Fog
8,1/1/2012 8:00,-1.4,-3.6,85,9,8.0,101.23,Fog
9,1/1/2012 9:00,-1.3,-3.1,88,15,4.0,101.20,Fog


In [18]:
final_df = df.drop(columns="Date/Time", axis=1, inplace=False)

In [19]:
#splitting into training and test sets.

from sklearn.model_selection import train_test_split


X = final_df.drop(columns='Weather Description', axis=1)
Y = final_df['Weather Description']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [20]:
#Normalization 
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [21]:
final_df.shape

(8784, 7)

# **2. Model Architecture:**

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


timesteps = 100  
n_features = 64 


model = Sequential()

model.add(LSTM(128, return_sequences=True, input_shape=(timesteps, n_features)))


model.add(LSTM(64))

model.add(Dense(1))


model.compile(loss='mse', optimizer='adam')  


model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 100, 128)          98816     
                                                                 
 lstm_3 (LSTM)               (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 148,289
Trainable params: 148,289
Non-trainable params: 0
_________________________________________________________________


# 3. Model Training:

In [32]:
import numpy as np


# Convert DataFrame to NumPy array
X_train_array = X_train.to_numpy()
y_train_array = y_train.to_numpy()

# Reshape X_train if necessary (e.g., if it's a 2D tensor)
X_train_array = X_train_array.reshape((num_samples, timesteps, features))

# Compile the LSTM model with appropriate loss function and optimizer
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model on the training set for a fixed number of epochs
history = model.fit(X_train_array, y_train_array, epochs=10, batch_size=32, validation_split=0.2)

# Monitor training/validation loss to ensure proper convergence


NameError: name 'num_samples' is not defined

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np


# Convert DataFrame to NumPy array
X_train_array = X_train.to_numpy()
y_train_array = y_train.to_numpy()


# Define model parameters
timesteps = 100  # Number of time steps in the sequence
n_features = 64  # Number of features in each time step

# Hyperparameter options for experimentation
lstm_layers = [1, 2, 3]  # Number of LSTM layers to try
lstm_units = [32, 64, 128]  # Units to try for each LSTM layer
dropout_rate = 0.2  # Dropout rate for regularization

# Loop through hyperparameter combinations
for n_lstm_layers in lstm_layers:
  for units_per_layer in lstm_units:
    # Create the model
    model = Sequential()

    # Add LSTM layers with dropout
    for _ in range(n_lstm_layers):
      if _ == 0:
        model.add(LSTM(units_per_layer, return_sequences=True, input_shape=(timesteps, n_features)))
      else:
        model.add(LSTM(units_per_layer, return_sequences=True))
      model.add(Dropout(dropout_rate))  # Add dropout after each LSTM layer

    # Add a dense output layer with 1 unit
    model.add(Dense(1))

    # Compile the model
    model.compile(loss='mse', optimizer='adam')  # Adjust loss and optimizer as needed

    

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')

    # Train the model
    history = model.fit(X_train_array, y_train_array, epochs=20, validation_split=0.2, callbacks=[early_stopping])

    # Evaluate the model (replace with your evaluation code)
    # ...

    # Print model details for each configuration
    print("Model with", n_lstm_layers, "LSTMs of", units_per_layer, "units each with", dropout_rate, "dropout")
    model.summary()


Epoch 1/20


ValueError: in user code:

    File "C:\Users\adity\anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\adity\anaconda3\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\adity\anaconda3\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\adity\anaconda3\lib\site-packages\keras\engine\training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\adity\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\adity\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 

In [35]:
# Assuming your data is currently a 2D array (samples, features)
X_train_reshaped = X_train_array.reshape((X_train.shape[0], timesteps, n_features))


ValueError: cannot reshape array of size 42162 into shape (7027,100,64)

In [36]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt


# 4. Model Evaluation:


In [ ]:

y_pred = model.predict(X_test)


mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

rmse = mean_squared_error(y_test, y_pred, squared=False) 
print("Root Mean Squared Error (RMSE):", rmse)


In [43]:
!pip install keras-tuner


In [42]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from kerastuner import RandomSearch

# Define model building function (replace with your actual model architecture)
def build_model(hp):
  model = Sequential()
  model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32), return_sequences=True, input_shape=(timesteps, n_features)))
  model.add(LSTM(units=hp.Int('units', min_value=16, max_value=64, step=16)))
  model.add(Dropout(rate=hp.Float('dropout', min_value=0.2, max_value=0.5)))
  model.add(Dense(1))
  model.compile(loss='mse', optimizer='adam')
  return model

tuner = TunedModel(
    build_model,
    tune_aggregates=['mean', 'std'],  # Track mean and standard deviation of metrics
    metric='val_loss',
    max_trials=10  # Adjust the number of trials
)

# Train the model with hyperparameter search
tuner.search(X_train_scaled, y_train, epochs=10, validation_split=0.2)

# Get the best model and its hyperparameters
best_model = tuner.get_best_models()[0]
best_hp = tuner.get_best_hyperparameters()[0]

# Evaluate the best model on the test set (replace with your code)
# ...

print("Best Hyperparameters:", best_hp.values)


NameError: name 'TunedModel' is not defined